<a href="https://colab.research.google.com/github/Yoon0527/AI_study/blob/main/%5BChapter_5%5D_Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 5. 오차역전파법
chapter 4에서 신경망의 가중치 매개변수의 기울기는 수치 미분을 사용해 구했다. 수치 미분은 단순하고 구현하기도 쉽지만 계산 시간이 오래 걸린다는 게 단점인데, 이번엔 가중치 매개변수의 기울기를 효율적으로 계산하는 오차역전파법(backpropagation)을 배워보자
> **오차역전파?**
> 
> 오차를 반대 방향으로 전파하는 방법이다.



In [4]:
# 곱셈 계층 
class MulLayer:
  def __init__(self):
    self.x = None
    self.y = None
  
  def forward(self, x, y):
    self.x = x
    self.y = y
    out = x*y

    return out

  def backward(self, dout):
    dx = dout * self.y # x와 y를 바꾼다.
    dy = dout * self.x

    return  dx, dy

* init(): 인스턴스 변수인 x와 y를 초기화한다. 이 두 변수는 순전파에서 입력값을 유지하기 위해서 사용한다.
* forward(): x와 y를 인수로 받고 두 값을 곱해서 반환한다. 
* backward(): 상류에서 넘어온 미분(dout)에 순전파 때의 값을 서로 바꿔 곱한 후 하류로 보낸다.

In [6]:
apple = 100
apple_num = 2
tax = 1.1

# 계층
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [9]:
# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


backward()가 받는 인수는 **순전파의 출력에 대한 미분**이다. mul_apple_layer라는 곱셈 계층은 순전파 때는 apple_price를 출력하지만, 역전파 때는 apple_price의 미분 값인 dapple_price를 인수로 받는다.

In [10]:
# 덧셈 계층
class AddLayer():
  def __init__(self):
    pass

  def forward(self, x, y):
    out = x+y
    return out

  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1
    return dx, dy

덧셈 계층에서는 초기화가 필요 없어서 init()에서 아무 일도 하지 않는다.
* forward(): 입력 받은 두 인수를 더해서 반환한다.
* backward(): 상류에서 내려온 미분을 그대로 하류로 흘린다.

In [12]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
pricee = mul_tax_layer.forward(all_price, tax)

# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)


220.00000000000003
110.00000000000001 2.2 3.3000000000000003 165.0 650


## 활성화 함수 구현

In [13]:
# ReLU
class ReLu:
  def __init__(self):
    self.mask = None

  def forward(self, x):
    self.mask = (x <= 0)
    out = x.copy()
    out[self.mask] = 0

    return out

  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout

    return dx

mask는 True, False로 구성된 넘파이 배열로, 순전파의 입력인 x의 원소 값이 0 이하인 인덱스는 True, 0보다 큰 원소는 False로 유지한다. 

In [14]:
# mask 변수는 다음 예와 같이 True/False로 구성된 넘파이 배열을 유지한다.
import numpy as np

x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

mask = (x<=0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]
